# Packages


In [ ]:
import sys
import os

# 假設 .venv 在專案根目錄
venv_path = os.path.abspath("../.venv/Lib/site-packages")  # Windows
# venv_path = os.path.abspath("../.venv/lib/python3.11/site-packages")  # Linux/Mac

if venv_path not in sys.path:
    sys.path.insert(0, venv_path)

print("✅ 已加入 .venv site-packages 到 sys.path")


In [1]:
# 📒 job_guardian_test.ipynb

# ------------------------------------------------------------
# 測試 0: 匯入 server.py
# ------------------------------------------------------------
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("server", "server.py")
server = importlib.util.module_from_spec(spec)
sys.modules["server"] = server
spec.loader.exec_module(server)

print("✅ server.py 匯入成功，MCP server 已載入。")


✅ server.py 匯入成功，MCP server 已載入。


In [2]:
# 用哪個PYTHON
import sys, platform, socket
print(sys.executable)
print(platform.system())
print(socket.getaddrinfo("mopsfin.twse.com.tw", 443))


/home/de-jung/projects/job-guardian/.venv/bin/python
Linux
[(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('117.56.218.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('117.56.218.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_RAW: 3>, 0, '', ('117.56.218.140', 443))]


# 測試爬蟲


In [3]:
# check certification
import sys, ssl, requests, certifi
print("Python:", sys.executable)
print("OpenSSL:", ssl.OPENSSL_VERSION)
print("Requests:", requests.__version__)
print("Certifi:", certifi.where())


Python: /home/de-jung/projects/job-guardian/.venv/bin/python
OpenSSL: OpenSSL 3.0.13 30 Jan 2024
Requests: 2.32.5
Certifi: /home/de-jung/projects/job-guardian/.venv/lib/python3.12/site-packages/certifi/cacert.pem


In [5]:
rows = server._fetch_csv_rows(server.ESG_URL)
print("筆數:", len(rows))
print("前5筆:", rows[0])


筆數: 850
前5筆: {'\ufeff出表日期': '1140924', '報告年度': '113', '公司代號': '1240', '公司名稱': '茂生農經', '員工福利平均數(仟元/人)': '982', '員工薪資平均數(仟元/人)': '841', '非擔任主管職務之全時員工薪資平均數(仟元/人)': '740', '非擔任主管之全時員工薪資中位數(仟元/人)': '649', '管理職女性主管佔比': '6.00%'}


In [6]:
# ------------------------------------------------------------
# 測試 1: 確認爬蟲來源可下載並解析
# ------------------------------------------------------------
for url in [server.ESG_URL, server.LAB_VIO_URL, server.GE_VIO_URL]:
    rows = server._fetch_csv_rows(url)
    print(f"來源: {url}")
    print(f"共 {len(rows)} 筆，前 1 筆：")
    print(rows[0] if rows else "⚠️ 無資料")
    print("="*80)


來源: https://mopsfin.twse.com.tw/opendata/t187ap46_O_5.csv
共 850 筆，前 1 筆：
{'\ufeff出表日期': '1140924', '報告年度': '113', '公司代號': '1240', '公司名稱': '茂生農經', '員工福利平均數(仟元/人)': '982', '員工薪資平均數(仟元/人)': '841', '非擔任主管職務之全時員工薪資平均數(仟元/人)': '740', '非擔任主管之全時員工薪資中位數(仟元/人)': '649', '管理職女性主管佔比': '6.00%'}
來源: https://apiservice.mol.gov.tw/OdService/download/A17000000J-030225-svj
共 68411 筆，前 1 筆：
{'主管機關': '南投縣', '公告日期': '20250829', '處分日期': '20250829', '處分字號': '府授勞資字第 1140202257號', '事業單位名稱或負責人': '賴慶鴻即萊緹蛋糕坊', '違法法規法條': '勞動基準法第24條', '違反法規內容': '延長工作時間未依規定加給工資', '罰鍰金額': '20000', '備註說明': ''}
來源: https://apiservice.mol.gov.tw/OdService/download/A17000000J-030226-sop
共 1064 筆，前 1 筆：
{'主管機關': '新竹縣', '公告日期': '20160105', '處分日期': '20151106', '處分字號': '104年府勞資字第1040137166號', '事業單位名稱或負責人': '千才科技股份有限公司(\n高嵩岳)', '違法法規法條': '性別工作平等法第7條', '違反法規內容': '招募特定性別', '罰鍰金額': '', '備註說明': ''}


# 測試工具內函式

In [8]:
# ------------------------------------------------------------
# 測試 2: 呼叫 esg_hr 工具
# ------------------------------------------------------------
result = server.esg_hr("茂生農經", year=113, limit=3)
print("esg_hr 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "count" in result
assert "source_url" in result
assert "meta" in result
print("✅ esg_hr output 格式正確")


esg_hr 測試輸出：
{'items': [], 'count': 0, 'source_url': 'https://mopsfin.twse.com.tw/opendata/t187ap46_O_5.csv', 'fetched_at': '2025-09-24T08:19:12Z', 'meta': {'query': '茂生農經', 'year': 113, 'partial_match': False}}
✅ esg_hr output 格式正確


In [ ]:
# ------------------------------------------------------------
# 測試 3: 呼叫 labor_violations 工具
# ------------------------------------------------------------
result = server.labor_violations("華碩", since_year=2020, limit=3)
print("labor_violations 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "stats" in result
assert "source_url" in result
assert "meta" in result
print("✅ labor_violations output 格式正確")


In [ ]:
# ------------------------------------------------------------
# 測試 4: 呼叫 ge_work_equality_violations 工具
# ------------------------------------------------------------
result = server.ge_work_equality_violations("華碩", since_year=2020, limit=3)
print("ge_work_equality_violations 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "stats" in result
assert "source_url" in result
assert "meta" in result
print("✅ ge_work_equality_violations output 格式正確")
